In [2]:
import pandas as pd
import sys
sys.path.append('../')
from evaluation.generated_dataset import GeneratedDataset

In [3]:
mp_20_source_sample = GeneratedDataset.from_transformations(("split", "test", "1k-sample"), sort=False).data

In [4]:
diffcsp_sample = GeneratedDataset.from_cache(("split", "test", "1k-sample", "WyckoffReduction", "DiffCSP++")).data
diffcsp_sample.index = mp_20_source_sample.index

In [5]:
matching_sg = (mp_20_source_sample.spacegroup_number == diffcsp_sample.spacegroup_number)
print(f"Matching spacegroup numbers (DiffCSP++): {matching_sg.sum()} out of {len(matching_sg)}")

Matching spacegroup numbers (DiffCSP++): 989 out of 1000


In [6]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()
structure_matches = pd.Series(
    [sm.fit(s1, s2) for s1, s2 in zip(diffcsp_sample.structure, mp_20_source_sample.structure)],
    index=diffcsp_sample.index
)

print(f"Matching structures (DiffCSP++): {structure_matches.sum()} out of {len(structure_matches)}")

Matching structures (DiffCSP++): 776 out of 1000


In [ ]:
import json
json_sample = diffcsp_sample.loc[~structure_matches].structure.map(lambda s: json.dumps(s.as_dict()))
json_sample.name = "json"
json_sample.to_csv("../generated/mp_20/split/test/1k-sample/WyckoffReduction/DiffCSP++/diffcsp_mismatches.csv.gz",
    index_label="mp_20_sequence_id")

In [8]:
cryspr_sample = GeneratedDataset.from_cache(("split", "test", "1k-sample", "WyckoffReduction", "CrySPR", "CHGNet_fix")).data

In [9]:
mp_20_source_sample = mp_20_source_sample.reset_index(drop=False)
mp_20_source_sub_sample = mp_20_source_sample.loc[mp_20_source_sample.index.isin(cryspr_sample.index)]

In [10]:
matching_sg = (mp_20_source_sub_sample.spacegroup_number == cryspr_sample.spacegroup_number)
print(f"Matching spacegroup numbers (CrySPR): {matching_sg.sum()} out of {len(matching_sg)}")

Matching spacegroup numbers (CrySPR): 773 out of 798


In [11]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()
structure_matches = pd.Series(
    [sm.fit(s1, s2) for s1, s2 in zip(cryspr_sample.structure, mp_20_source_sub_sample.structure)],
    index=cryspr_sample.index
)

print(f"Matching structures (CrySPR): {structure_matches.sum()} out of {len(structure_matches)}")

Matching structures (CrySPR): 561 out of 798
